In [7]:
### This is a starting example for the ELAIS-QST mini project


#Implementation of the protocol from https://arxiv.org/pdf/2007.13126.pdf
#Honest verifier and line graph
#Advantage for the honest verifier and fcg setting : generator testing completely suffices

%matplotlib inline
import numpy as np
# Importing standard Qiskit libraries and configuring account
import qiskit
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math as math
# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import depolarizing_error
# Loading your IBM Q account(s)

##### TODO : load your account, choose another backend
##### Feel free to experiment with the real world hardware they have available. Better look on the topology of the quantum computer of your choice
##### People nowadays often just use the transpiler but checking which qubit has what error rate, how you map your graph etc can be useful.
##### Topology of quantum computers is a big deal. Generally I prefer fully configurable QC hardware which is indeed an advantage of some
##### QC hardware but if the topology is not fully configurable the circuit must be mapped somehow and how precisely the gates are mapped
##### can matter a lot :-)

##### This is not a problem in the simulator but generally I'd advise to look at it. Example : You have a quantum computer which
##### consists of a line of qubits which have nearest neighbor connectivity. Testing a fully connected graph on this hardware makes 
##### less sense than having a fully connected topology because to implement it on the line the transpiler needs to use a lot of 
##### swap gates to even get the circuit on the hardware.

provider = IBMQ.load_account()

simulator2 = Aer.get_backend('statevector_simulator')

#select graph structure by giving the adjacency matrix

##### Example adjacency matrix - a line graph. The first qubit is connected to the second, the second to the first and third, and the
##### third only to the second (the middle node in the graph).
adj_matrix = [[0,1,0],[1,0,1],[0,1,0]]


##### some helper function for processing the result from Qiskit
def Convert(string): 
    list1=[] 
    list1[:0]=string 
    return list1


##### Here I only test the generators. The other tests are more diffcult and I calculated the stabilizer tables by hand and then hardcoded the
##### circuit. But testing the generators is easy. You have X on the diagonal. You can measure in the computational basis. That is the Z measurement
##### For measuring identity you just say pass.


def stabilizer_test(circuit,num_qubits,alpha):
    ####### This function generates the generator stabilizers from the adjacency matrix
    #num_qubits = number of qubits in the graph (or dimension of the adjacency matrix)
    #alpha : which generator is measured? The first one would be alpha = 0. 
    #For the line graph that would be XZI. X on the diagonal, Z for the one connection and I because the first qubit is not connected to the last one.
    for i in range(0,num_qubits):
        if i == alpha:
            #diagonal element = X, so apply the Hadamard gate to measure X
            circuit.h(i)
            circuit.measure(i,i)
        if adj_matrix[alpha][i] == 0:
            #If entry of adjacency matrix is zero (= no connection present), measure identity. This corresponds to "do nothing, return zero"
            pass
        if adj_matrix[alpha][i] == 1:
            #If entry of adjacency matrix is one (= connection present), measure Z. This is just the normal measurement
            circuit.measure(i,i)
    return(circuit)

def stabilizer_test_with_depolarizing_noise_line(noise_param,J,m):
    ##### Noise parameter is the noise parameter for white noise. If you test on actual quantum hardware, you need to modify this function.
    ##### m is a parameter I calculated by hand. The verification protocols all use the main structure - generate X copies of the unknown state, measure
    ##### a part of them. Depending on how many copies you measure and some statistics you can get different statements about the fidelity of 
    ##### the state. The idea is that you deal with a cheating source (and cheating parties in the network) and the source wants to give you a
    ##### wrong state. As the source does not know which state is the one you test, it cannot necessarily tell when it should send you the good or the
    ##### bad state. Here m comes directly from the protocol. As our scenario is different - honest source and only noise, I scaled m to give
    ##### me as many copies as I could get with the number of shots available. The number of tests depends on the number of qubits (here called J)
    ##### but the quantum computer I used only had 8192 shots available. To get as many shots as possible I selected m to appoach num_tests depending
    ##### on J to be as close to 8192 as possible. 
    n_pass = 0
    num_tests = math.ceil(m*J**4*math.log(J)) ###### this comes directly from the protocol
    num_qubits = J
    for alpha in range(J):
        for beta in range(num_tests):
            ### white noise model/depolarizing channel
            graph_state_circuit = qiskit.circuit.library.GraphState(adj_matrix) ###### generate the graph state circuit
            new_creg = graph_state_circuit._create_creg(len(graph_state_circuit.qubits), 'meas') ##### generate register formeasuring
            graph_state_circuit.add_register(new_creg)
            graph_state_circuit.barrier()
            qiskit.providers.aer.noise.depolarizing_error(noise_param,num_qubits) ##### add depolarizing noise
            noise_model = NoiseModel()
            ###### depending on whether you want to add one qubit noise or two qubit noise (these are the gates that add entanglement)
            ###### comment out the corresponding lines
            error = depolarizing_error(noise_param,num_qubits=1) #1qb noise 
            error2 = depolarizing_error(noise_param,num_qubits=2)#2qb noise
            noise_model.add_all_qubit_quantum_error(error,['h']) #belongs to 1qb noise, comment out for the two qubit noise part
            noise_model.add_all_qubit_quantum_error(error2,['cz']) ### belongs to 2 qb noise, comment out for the 1 qb noise part
            basis_gates = noise_model.basis_gates
            graph_state_circuit = stabilizer_test(graph_state_circuit,num_qubits,alpha) # this calls the function and adds the stabilizer test
            simulator = Aer.get_backend('qasm_simulator') #backend of your choice here
            shots = 1 #number of shots
            result = execute(graph_state_circuit,simulator,basis_gates=basis_gates,shots=shots,noise_model=noise_model).result() #execute the circuit
            counts = result.get_counts(graph_state_circuit) #get the result. The result is a binary string
            #if product over all returns equals 1, accept
            ###### this is for converting the result. I had no idea on how to work with dictionaries, so I iterate over the dictionary to add the results.
            dummy = list(counts.values())
            j = 0
            #countslist.append(counts)
            for i in list(counts):
                string_i = Convert(i) #represent element as list of strings to iterate over returned element 0000 = ['0','0','0','0']
                brauchbarer_string_i = [int(x) for x in string_i] #### helper variable for converting the dictionary
                #### sum of the results needs to be dividable by 2. This is equal to saying that all measurement results are equal to +1.
                #### Physicists use +1 for a passed stabilizer test and -1 for a non passed one. The product of the results must be 1.
                #### In the language of quantum computers, a passed result equals 0 (so +1 in physics = 0 in quantum computing and -1 in physics
                #### equals 1 in quantum computing. In this notation : product of all results must be 1 becomes : string returned by the
                #### QC must be even. Or divisable by 2 with rest zero. Or the string has an even Hamming weight (numbers of 1 in string). 
                #### That is the 
                if np.sum(brauchbarer_string_i) % 2 == 0:
                    n_pass += int(dummy[j])
                j += 1
    return n_pass/(num_tests*J) #### I came up with this metric by some arithmetic. Details can be seen in the master thesis. I prefer this metric as it gives you a percentage of tests to be passed instead of some strange uncomparable number.



ibmqfactory.load_account:WARNING:2022-02-08 14:58:57,547: Credentials are already in use. The existing account in the session will be replaced.


In [12]:
##### Repeat result for a different noise parameter . Use a certain number of shots and then repeat experiment 20 times for statistics. But the 
##### standard deviation is so low that you even don't need the statistics. I just found it nicer to gather some statistics and then always repeated the 
##### 20 times for consistency with the first test runs of the experiment on real quantum hardware.
noise_parameter = [0.05 for i in range(20)]

percentage_passed = [stabilizer_test_with_depolarizing_noise_line(noise_param,3,23) for noise_param in noise_parameter] #23 was the m I calculated for J=3. The m values can be seen in the master thesis.
##### Feel free to deviate when testing graphs with more than 5 qubits.

print(percentage_passed) #the list with the actual values from 20 experiments
print(np.mean(percentage_passed),np.std(percentage_passed)) #mean and standard deviation of experiment results.


[0.9099495196222114, 0.9135319980459209, 0.9182543559680834, 0.9091353199804592, 0.9140205178309722, 0.9140205178309722, 0.9146718775443738, 0.9053900016283993, 0.9211854746783912, 0.9086468001954079, 0.9086468001954079, 0.9241165933886989, 0.9158117570428269, 0.9149975574010747, 0.9163002768278782, 0.9138576779026217, 0.9135319980459209, 0.9146718775443738, 0.9151603973294252, 0.91320631818922]
0.9139553818596321 0.004221733099109159
